In [151]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras import regularizers

from dummies_bins_test_train_cv import get_Xy_train_test, cross_validation_process
# from clean_chess_game_log import main_cleanup
# _,_,_ = main_cleanup('../data/dest.pgn')

In [152]:
df = pd.read_csv('../data/use_for_predictions.csv')

In [182]:
X_train, X_test, y_train, y_test, X, y, df_clean = get_Xy_train_test(df, .985, .995)

y Shape: (2098,)
X Shape: (2098, 29)
X_train Shape: (2076, 29)
X_test Shape: (22, 29)
y_train Shape: (2076,)
y_test Shape: (22,)


In [183]:
df_clean.columns

Index(['game_time', 'color', 'diff_bin', 'st_1', 'st_2', 'st_6', 'st_7',
       'st_8', 'st_9', 'st_10', 'st_11', 'st_12', 'st_13', 'st_14', 'st_15',
       'st_16', 'st_17', 'st_18', 'st_19', 'st_20', 'st_21', 'st_22', 'st_23',
       'wd_1', 'wd_2', 'wd_3', 'wd_4', 'wd_5', 'wd_6'],
      dtype='object')

In [184]:
X_train = X_train.astype('float64')
X_test = X_test.astype('float64')
y_train = y_train.astype('int64')
y_test = y_test.astype('int64')

In [185]:
X_train.shape

(2076, 29)

In [186]:
X_test.shape

(22, 29)

In [187]:
y_train.shape

(2076,)

In [188]:
std_sclr = StandardScaler()
X_train = std_sclr.fit_transform(X_train)
X_test = std_sclr.fit_transform(X_test)

In [189]:
std_sclr.scale_

array([1.        , 0.49166608, 9.03219677, 1.        , 1.        ,
       1.        , 0.28747979, 1.        , 0.4190702 , 1.        ,
       0.2082989 , 0.34317429, 0.28747979, 0.2082989 , 0.28747979,
       0.2082989 , 0.28747979, 0.28747979, 1.        , 0.2082989 ,
       1.        , 1.        , 1.        , 0.38569461, 0.34317429,
       0.28747979, 0.44536177, 0.28747979, 0.28747979])

In [190]:
def _classifier():
    classifier = Sequential()

    classifier.add(Dense(units=128, kernel_initializer='uniform',
                         activation='softmax', input_dim=29))

    classifier.add(
        Dense(units=256, kernel_initializer='normal', activation='relu'))

    classifier.add(
        Dense(units=64, kernel_initializer='uniform', activation='softmax'))

    classifier.add(
        Dense(units=1, kernel_initializer='normal', activation='sigmoid'))

    classifier.compile(
        optimizer='nadam', loss='binary_crossentropy', metrics=['binary_accuracy'])

    return classifier

In [191]:
classifier = _classifier()

In [192]:
classifier.fit(X_train, y_train, batch_size=6, epochs=100,
               class_weight='balanced', shuffle=False)

Epoch 1/100
2076/2076 [==============================] - 4s 2ms/step - loss: 0.6816 - binary_accuracy: 0.5568
Epoch 2/100
2076/2076 [==============================] - 2s 920us/step - loss: 0.6141 - binary_accuracy: 0.6965
Epoch 3/100
2076/2076 [==============================] - 2s 892us/step - loss: 0.5967 - binary_accuracy: 0.7047
Epoch 4/100
2076/2076 [==============================] - 2s 949us/step - loss: 0.5845 - binary_accuracy: 0.7115
Epoch 5/100
2076/2076 [==============================] - 2s 900us/step - loss: 0.5751 - binary_accuracy: 0.7134
Epoch 6/100
2076/2076 [==============================] - 2s 912us/step - loss: 0.5677 - binary_accuracy: 0.7153
Epoch 7/100
2076/2076 [==============================] - 2s 927us/step - loss: 0.5596 - binary_accuracy: 0.7110
Epoch 8/100
2076/2076 [==============================] - 2s 918us/step - loss: 0.5522 - binary_accuracy: 0.7129
Epoch 9/100
2076/2076 [==============================] - 2s 1ms/step - loss: 0.5462 - binary_accuracy: 0.7

2076/2076 [==============================] - 2s 859us/step - loss: 0.3850 - binary_accuracy: 0.8502
Epoch 74/100
2076/2076 [==============================] - 2s 831us/step - loss: 0.3864 - binary_accuracy: 0.8497
Epoch 75/100
2076/2076 [==============================] - 2s 853us/step - loss: 0.3831 - binary_accuracy: 0.8550
Epoch 76/100
2076/2076 [==============================] - 2s 858us/step - loss: 0.3824 - binary_accuracy: 0.8536
Epoch 77/100
2076/2076 [==============================] - 2s 859us/step - loss: 0.3777 - binary_accuracy: 0.8526
Epoch 78/100
2076/2076 [==============================] - 2s 860us/step - loss: 0.3757 - binary_accuracy: 0.8536
Epoch 79/100
2076/2076 [==============================] - 2s 866us/step - loss: 0.3677 - binary_accuracy: 0.8622
Epoch 80/100
2076/2076 [==============================] - 2s 847us/step - loss: 0.3710 - binary_accuracy: 0.8579
Epoch 81/100
2076/2076 [==============================] - 2s 871us/step - loss: 0.3767 - binary_accuracy: 0.8

In [193]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [194]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[7, 3],
       [3, 9]])

In [195]:
print(f'{((cm[0][0]+cm[1][1])/cm.sum()*100).round(1)}%')

72.7%


In [197]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_108 (Dense)            (None, 128)               3840      
_________________________________________________________________
dense_109 (Dense)            (None, 256)               33024     
_________________________________________________________________
dense_110 (Dense)            (None, 64)                16448     
_________________________________________________________________
dense_111 (Dense)            (None, 1)                 65        
Total params: 53,377
Trainable params: 53,377
Non-trainable params: 0
_________________________________________________________________
